In [1]:
from osgeo import gdal, ogr
from osgeo.gdalconst import *
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
gdal.PushErrorHandler('CPLQuietErrorHandler')

0

# Critical loads

The critical loads workflow needs updating this year. This notebook documents some intiial exploration.

## 1. Vegetation map

One operation that has previously taken a lot of time is estimating exceedances based on vegetation cover. As a test, I'd like to calculate summary land use statistics for each cell in the BLR grid. If this works reasonably quickly, we should be able to do everything else we need fairly easily.

To begin with, I'll run tests uisng the 60 m resolution data, as this is easier to manipulate initially.

### 1.1. Mosiac to a single dataset

The raw, 60 m vegetation data is here:

K:\Avdeling\317 Klima- og miljømodellering\KAU\Focal Centre\Vegetation\Veg map\satveg_30\1

I've copied this locally and used the `Mosiac_To_New_Raster` tool in ArcToolbox to combine the tiles into a single 8-bit integer GeoTiff (`sat_veg_60m_all.tif`). This file has an uncompressed size of 557 MB (which implies the 30 m dataset will be about 2.2 GB). We should be able to work with these grids fairly efficiently, but note that if I need to reclassify the land uses to decimal values for the critical loads calculations, the file sizes will increase by a factor of 4, as they will need to be upcast to 32-bit floats.

For now, I will focus on calculating some simple zonal statistics, as this will give an indication of performance.

### 1.2. Read land use lookup table

In [2]:
# Read lookup table
in_xlsx = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\sat_veg_land_use_classes.xlsx')
df = pd.read_excel(in_xlsx, sheetname='EUNIS_tilGIS', index_col=0)

# Get cols of interest
df = df[['EUNISveg',]]

df

,EUNISveg
NORUTcode,
1,Coniferous woodland
2,Mixed taiga woodland with Betula
3,Pine taiga woodland
4,Broadleaved deciduous woodland
5,Broadleaved deciduous woodland
6,Eurasian boreal Betula woods
7,Eurasian boreal Betula woods
8,Eurasian boreal Betula woods
9,Raised- and blanket bogs


### 1.3. Reproject BLR grid

The BLR grid is stored in WGS84 geographic co-ordinates, whereas the vegetation data is all in UTM Zone 33N projected co-ordinates (based on the WGS84 datum). I have therefore reprojected the BLR grid to match the ratser data, as this will improve performance by eliminating the need for "on-the-fly" reprojection. The reprojected dataset is here:

C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS\Shapefiles\blrgrid_uten_grums_utm_z33n.shp

### 1.4. Zonal statistics code

The code below is modified from [here](https://gist.github.com/perrygeo/5667173) and provides low-level access to GDAL, which should be substantially faster than ArcGIS.

In [3]:
def remap_categories(category_map, stats):
    """ Modified from https://gist.github.com/perrygeo/5667173
        Original code copyright 2013 Matthew Perry
    """
    def lookup(m, k):
        """ Dict lookup but returns original key if not found
        """
        try:
            return m[k]
        except KeyError:
            return k

    return {lookup(category_map, k): v
            for k, v in stats.items()}

def bbox_to_pixel_offsets(gt, bbox):
    """ Modified from https://gist.github.com/perrygeo/5667173
        Original code copyright 2013 Matthew Perry
    """
    originX = gt[0]
    originY = gt[3]
    pixel_width = gt[1]
    pixel_height = gt[5]
    x1 = int((bbox[0] - originX) / pixel_width)
    x2 = int((bbox[1] - originX) / pixel_width) + 1

    y1 = int((bbox[3] - originY) / pixel_height)
    y2 = int((bbox[2] - originY) / pixel_height) + 1

    xsize = x2 - x1
    ysize = y2 - y1
    
    return (x1, y1, xsize, ysize)

def zonal_stats(vector_path, raster_path, nodata_value=None, global_src_extent=False,
                categorical=False, category_map=None):
    """ Modified from https://gist.github.com/perrygeo/5667173
        Original code copyright 2013 Matthew Perry
    """
    rds = gdal.Open(raster_path, GA_ReadOnly)
    assert(rds)
    rb = rds.GetRasterBand(1)
    rgt = rds.GetGeoTransform()

    if nodata_value:
        nodata_value = float(nodata_value)
        rb.SetNoDataValue(nodata_value)

    vds = ogr.Open(vector_path, GA_ReadOnly)  # TODO maybe open update if we want to write stats
    assert(vds)
    vlyr = vds.GetLayer(0)

    # create an in-memory numpy array of the source raster data
    # covering the whole extent of the vector layer
    if global_src_extent:
        # use global source extent
        # useful only when disk IO or raster scanning inefficiencies are your limiting factor
        # advantage: reads raster data in one pass
        # disadvantage: large vector extents may have big memory requirements
        src_offset = bbox_to_pixel_offsets(rgt, vlyr.GetExtent())
        src_array = rb.ReadAsArray(*src_offset)

        # calculate new geotransform of the layer subset
        new_gt = (
            (rgt[0] + (src_offset[0] * rgt[1])),
            rgt[1],
            0.0,
            (rgt[3] + (src_offset[1] * rgt[5])),
            0.0,
            rgt[5]
        )

    mem_drv = ogr.GetDriverByName('Memory')
    driver = gdal.GetDriverByName('MEM')

    # Loop through vectors
    stats = []
    feat = vlyr.GetNextFeature()
    while feat is not None:

        if not global_src_extent:
            # use local source extent
            # fastest option when you have fast disks and well indexed raster (ie tiled Geotiff)
            # advantage: each feature uses the smallest raster chunk
            # disadvantage: lots of reads on the source raster
            src_offset = bbox_to_pixel_offsets(rgt, feat.geometry().GetEnvelope())
            src_array = rb.ReadAsArray(*src_offset)

            # calculate new geotransform of the feature subset
            new_gt = (
                (rgt[0] + (src_offset[0] * rgt[1])),
                rgt[1],
                0.0,
                (rgt[3] + (src_offset[1] * rgt[5])),
                0.0,
                rgt[5]
            )

        # Create a temporary vector layer in memory
        mem_ds = mem_drv.CreateDataSource('out')
        mem_layer = mem_ds.CreateLayer('poly', None, ogr.wkbPolygon)
        mem_layer.CreateFeature(feat.Clone())

        # Rasterize it
        rvds = driver.Create('', src_offset[2], src_offset[3], 1, gdal.GDT_Byte)
        rvds.SetGeoTransform(new_gt)
        gdal.RasterizeLayer(rvds, [1], mem_layer, burn_values=[1])
        rv_array = rvds.ReadAsArray()

        # Mask the source data array with our current feature
        # we take the logical_not to flip 0<->1 to get the correct mask effect
        # we also mask out nodata values explictly
        masked = np.ma.MaskedArray(
            src_array,
            mask=np.logical_or(
                src_array == nodata_value,
                np.logical_not(rv_array)
            )
        )

        if categorical:
            # Get cell counts for each category
            keys, counts = np.unique(masked.compressed(), return_counts=True)
            pixel_count = dict(zip([np.asscalar(k) for k in keys],
                                   [np.asscalar(c) for c in counts]))

            feature_stats = dict(pixel_count)
            if category_map:
                feature_stats = remap_categories(category_map, feature_stats)
        
        else:
            # Get summary stats
            feature_stats = {
                'min': float(masked.min()),
                'mean': float(masked.mean()),
                'max': float(masked.max()),
                'std': float(masked.std()),
                'sum': float(masked.sum()),
                'count': int(masked.count()),
                'fid': int(feat.GetFID())}                        
                        
        stats.append(feature_stats)

        rvds = None
        mem_ds = None
        feat = vlyr.GetNextFeature()

    vds = None
    rds = None
    
    return pd.DataFrame(stats)

### 1.5. Run zonal statistics

First, convert the Excel land cover classes to a dictionary that maps the land use codes to class names.

In [4]:
# Get mapping of LU codes to land classes
category_map = df.to_dict()['EUNISveg']

Now we can run the zonal stats.

In [5]:
%%time

# Input datasets
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Shapefiles\blrgrid_uten_grums_utm_z33n.shp')

in_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Raster\sat_veg_60m_all.tif')

# Zonal stats
zs = zonal_stats(in_shp, in_tif, categorical=True, 
                 category_map=category_map)

# Convert cell counts to areas in km2
zs = zs*60.*60./1.E6

Wall time: 5.69 s


C:\Data\WinPython-64bit-2.7.10.3\python-2.7.10.amd64\lib\site-packages\ipykernel\__main__.py:116: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


The zonal stats processing takes just under 6 seconds, which implies we should also be able to process the 30 m dataset without too much difficulty (although 30 m seems unnecessary if we're interested in the whole of Norway).

### 1.6. Combine with BLR data

In [6]:
# Read BLR grid
blr = gpd.read_file(in_shp)

# Convert blr area to km2
blr['blr_area_km2'] = blr['area_m2'] / 1.E6

# Delete unwanted cols
del blr['geometry'], blr['area_m2']

# Convert to df
blr = pd.DataFrame(blr)

# Rename cols
blr.columns = ['blr_id', 'blr_area_km2']

# Join
res = blr.join(zs)

# Melt to long format
res = pd.melt(res, id_vars=['blr_id', 'blr_area_km2'], 
              var_name='land_use',
              value_name='area_km2')

# Set multi-index
res.sort_values(['blr_id', 'land_use'], inplace=True)
res.set_index(['blr_id', 'land_use'], inplace=True)

res.head(40)

blr_area_km2  \
blr_id   land_use                                                           
58005004 Alpine and subalpine acid grasslands                   54.103222   
         Arable land and market gardens                         54.103222   
         Arctic, alpine and sub-alpine scrub habitats           54.103222   
         Boreo-alpine acidocline snow-patch grassland an...     54.103222   
         Broadleaved deciduous woodland                         54.103222   
         Coniferous woodland                                    54.103222   
         Eurasian boreal Betula woods                           54.103222   
         Inland surface water habitats                          54.103222   
         Mixed taiga woodland with Betula                       54.103222   
         Moss and lichen dominated mountain summits             54.103222   
         Open non-Mediterranean dry acid and neutral gra...     54.103222   
         Pine taiga woodland                                    54.103222   
         Raised- and blanket bogs                               54.103222   
         Snow or ice-dominated habitats                         54.103222   
         Temperate shrub habitats: wet and dry heaths           54.103222   
         Urban areas                                            54.103222   
         Valley mires, poor fens and transition mires           54.103222   
         Y                                                      54.103222   
         ingen                                                  54.103222   
58006001 Alpine and subalpine acid grasslands                  193.928481   
         Arable land and market gardens                        193.928481   
         Arctic, alpine and sub-alpine scrub habitats          193.928481   
         Boreo-alpine acidocline snow-patch grassland an...    193.928481   
         Broadleaved deciduous woodland                        193.928481   
         Coniferous woodland                                   193.928481   
         Eurasian boreal Betula woods                          193.928481   
         Inland surface water habitats                         193.928481   
         Mixed taiga woodland with Betula                      193.928481   
         Moss and lichen dominated mountain summits            193.928481   
         Open non-Mediterranean dry acid and neutral gra...    193.928481   
         Pine taiga woodland                                   193.928481   
         Raised- and blanket bogs                              193.928481   
         Snow or ice-dominated habitats                        193.928481   
         Temperate shrub habitats: wet and dry heaths          193.928481   
         Urban areas                                           193.928481   
         Valley mires, poor fens and transition mires          193.928481   
         Y                                                     193.928481   
         ingen                                                 193.928481   
58006002 Alpine and subalpine acid grasslands                  204.411417   
         Arable land and market gardens                        204.411417   

                                                             area_km2  
blr_id   land_use                                                      
58005004 Alpine and subalpine acid grasslands                  4.8312  
         Arable land and market gardens                        2.6568  
         Arctic, alpine and sub-alpine scrub habitats          6.2892  
         Boreo-alpine acidocline snow-patch grassland an...    1.0980  
         Broadleaved deciduous woodland                        0.8064  
         Coniferous woodland                                   0.4896  
         Eurasian boreal Betula woods                          0.4212  
         Inland surface water habitats                         4.0932  
         Mixed taiga woodland with Betula                      0.8604  
         Moss and lichen dominated

In [7]:
# Write to CSV
out_csv = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\blr_land_use_props.csv')
res.to_csv(out_csv)